In [7]:
from typing import Dict, Tuple, Optional, List, Any, Dict
from geopy.distance import geodesic
from geopy.geocoders import Nominatim
import pandas as pd
from datetime import datetime
import numpy as np
import time
import os
import logging
import requests

from enum import Enum
from dataclasses import dataclass


# Configuração do logger
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

In [8]:


class TipoTabela(Enum):
    LOTACAO = "A"
    AUTOMOTOR = "B"
    ALTO_DESEMPENHO = "C"
    AUTOMOTOR_ALTO_DESEMPENHO = "D"

class TipoCargaANTT(Enum):
    GRANEL_SOLIDO = 1
    GRANEL_LIQUIDO = 2
    FRIGORIFICADA = 3
    CONTEINERIZADA = 4
    CARGA_GERAL = 5
    NEOGRANEL = 6
    PERIGOSA_GRANEL_SOLIDO = 7
    PERIGOSA_GRANEL_LIQUIDO = 8
    PERIGOSA_FRIGORIFICADA = 9
    PERIGOSA_CONTEINERIZADA = 10
    PERIGOSA_CARGA_GERAL = 11
    GRANEL_PRESSURIZADA = 12

@dataclass
class CoeficientesANTT:
    """Coeficientes de custo conforme ANTT"""
    deslocamento: float  # CCD - R$/km
    carga_descarga: float  # CC - R$

class TabelaANTT:
    def __init__(self) -> None:
        """Inicializa as tabelas de coeficientes ANTT"""
        self.tabelas: Dict[TipoTabela, Dict[int, Dict[int, CoeficientesANTT]]] = {
            TipoTabela.LOTACAO: self._init_tabela_lotacao(),
            TipoTabela.AUTOMOTOR: self._init_tabela_automotor(),
            TipoTabela.ALTO_DESEMPENHO: self._init_tabela_alto_desempenho(),
            TipoTabela.AUTOMOTOR_ALTO_DESEMPENHO: self._init_tabela_automotor_alto_desempenho()
        }

    def _init_tabela_lotacao(self) -> Dict[int, Dict[int, CoeficientesANTT]]:
        """Inicializa Tabela A - Transporte Rodoviário de Carga Lotação"""
        return {
            TipoCargaANTT.GRANEL_SOLIDO.value: {
                2: CoeficientesANTT(3.4880, 408.57),
                3: CoeficientesANTT(4.4505, 495.60),
                4: CoeficientesANTT(5.1405, 536.30),
                5: CoeficientesANTT(5.5749, 520.16),
                6: CoeficientesANTT(6.2063, 546.66),
                7: CoeficientesANTT(7.0506, 710.36),
                9: CoeficientesANTT(7.9437, 755.95)
            },
            TipoCargaANTT.CARGA_GERAL.value: {
                2: CoeficientesANTT(3.4645, 402.10),
                3: CoeficientesANTT(4.4360, 491.61),
                4: CoeficientesANTT(5.0845, 520.92),
                5: CoeficientesANTT(5.5455, 512.07),
                6: CoeficientesANTT(6.1249, 524.27),
                7: CoeficientesANTT(7.0577, 712.31),
                9: CoeficientesANTT(7.9940, 769.78)
            }
            # Adicionar outros tipos de carga conforme necessário
        }

    def _init_tabela_automotor(self) -> Dict[int, Dict[int, CoeficientesANTT]]:
        """Inicializa Tabela B - Operações com Veículo Automotor"""
        return {
            TipoCargaANTT.CARGA_GERAL.value: {
                4: CoeficientesANTT(4.5341, 460.91),
                5: CoeficientesANTT(5.0170, 489.62),
                6: CoeficientesANTT(5.8615, 574.70),
                7: CoeficientesANTT(6.0784, 608.39),
                9: CoeficientesANTT(6.6315, 623.47)
            }
            # Adicionar outros tipos de carga
        }

    def _init_tabela_alto_desempenho(self) -> Dict[int, Dict[int, CoeficientesANTT]]:
        """Inicializa Tabela C - Transporte de Alto Desempenho"""
        return {
            TipoCargaANTT.CARGA_GERAL.value: {
                2: CoeficientesANTT(2.8980, 147.78),
                3: CoeficientesANTT(3.6608, 167.06),
                4: CoeficientesANTT(4.3135, 185.33),
                5: CoeficientesANTT(4.8526, 192.93),
                6: CoeficientesANTT(5.4943, 210.52),
                7: CoeficientesANTT(5.8755, 232.32),
                9: CoeficientesANTT(6.6943, 247.40)
            }
            # Adicionar outros tipos de carga
        }

    def _init_tabela_automotor_alto_desempenho(self) -> Dict[int, Dict[int, CoeficientesANTT]]:
        """Inicializa Tabela D - Automotor de Alto Desempenho"""
        return {
            TipoCargaANTT.CARGA_GERAL.value: {
                4: CoeficientesANTT(3.9029, 172.40),
                5: CoeficientesANTT(4.3189, 178.59),
                6: CoeficientesANTT(4.9651, 196.92),
                7: CoeficientesANTT(5.1384, 209.93),
                9: CoeficientesANTT(5.6727, 215.88)
            }
            # Adicionar outros tipos de carga
        }

    def obter_coeficientes(self, 
                          tipo_tabela: TipoTabela,
                          tipo_carga: TipoCargaANTT, 
                          num_eixos: int) -> Optional[CoeficientesANTT]:
        """Obtém os coeficientes para os parâmetros especificados"""
        try:
            return self.tabelas[tipo_tabela][tipo_carga.value][num_eixos]
        except KeyError:
            return None

    def calcular_frete_base(self,
                           tipo_tabela: TipoTabela,
                           tipo_carga: TipoCargaANTT,
                           num_eixos: int,
                           distancia: float) -> Optional[float]:
        """Calcula o frete base conforme tabela ANTT"""
        coef = self.obter_coeficientes(tipo_tabela, tipo_carga, num_eixos)
        if not coef:
            return None
        return (coef.deslocamento * distancia) + coef.carga_descarga



class CalculadoraFrete:
    def __init__(self) -> None:
        """
        Inicializa a calculadora de frete com o geocodificador e todos os fatores
        """
        self.logger = logging.getLogger(__name__)
        self.osrm_url = "http://router.project-osrm.org/route/v1/driving"
        self.geocoder = Nominatim(user_agent="minha_calculadora_frete")
        self.cache_coordenadas: Dict[str, Tuple[float, float]] = {}
        
        # Ajuste dos multiplicadores para valores mais realistas
        self.fatores_prazo = {
            "normal": 1.0,
            "expresso": 1.2,  
            "urgente": 1.5    
        }
        
        self.fatores_tipo_carga = {
            "comum": 1.0,
            "frágil": 1.2,    
            "perecível": 1.3, 
            "perigosa": 1.5   
        }
        
        self.fatores_regiao = {
            "Norte": 1.2,     
            "Nordeste": 1.15, 
            "Centro-Oeste": 1.1,  
            "Sudeste": 1.0,
            "Sul": 1.05       
        }

        self.faixas_desconto_distancia = {
            500: 0.0,    # Até 500km: sem desconto
            1000: 0.1,   # 501-1000km: 10% desconto
            2000: 0.2,   # 1001-2000km: 20% desconto
            3000: 0.25,  # 2001-3000km: 25% desconto
            float('inf'): 0.3  # Acima de 3000km: 30% desconto
        }

        # Fator de cubagem padrão do mercado
        self.fator_cubagem = 300  # Divisor para peso cubado (padrão mercado)

        # Custos médios de pedágio por região
        self.custos_pedagio = {
            "Sudeste": 0.15,  # R$ 0,15 por km
            "Sul": 0.12,
            "Nordeste": 0.08,
            "Centro-Oeste": 0.10,
            "Norte": 0.05
        }

         # Inicializa a tabela ANTT
        self.tabela_antt = TabelaANTT()
        
        # Mapeamento de tipos de carga para ANTT
        self.mapa_tipo_carga_antt = {
            "comum": TipoCargaANTT.CARGA_GERAL,
            "frágil": TipoCargaANTT.CARGA_GERAL,
            "perecível": TipoCargaANTT.FRIGORIFICADA,
            "perigosa": TipoCargaANTT.PERIGOSA_CARGA_GERAL
        }


        
        # Mapeamento de cidades para regiões
        self.mapa_regioes = {
            "São Paulo": "Sudeste",
            "Rio de Janeiro": "Sudeste",
            "Belo Horizonte": "Sudeste",
            "Vitória": "Sudeste",
            "Curitiba": "Sul",
            "Florianópolis": "Sul",
            "Porto Alegre": "Sul",
            "Salvador": "Nordeste",
            "Recife": "Nordeste",
            "Fortaleza": "Nordeste",
            "Manaus": "Norte",
            "Belém": "Norte",
            "Brasília": "Centro-Oeste",
            "Goiânia": "Centro-Oeste",
            "Cuiabá": "Centro-Oeste"
        }

        # Fatores sazonais por mês
        self.fatores_sazonais = {
            1: 1.1,   # Janeiro (alta demanda)
            2: 1.1,   # Fevereiro (carnaval)
            3: 1.0,
            4: 1.0,
            5: 1.0,
            6: 1.0,
            7: 1.0,
            8: 1.0,
            9: 1.0,
            10: 1.0,
            11: 1.2,  # Novembro (black friday)
            12: 1.3   # Dezembro (natal)
        }
        # Adicionar restrições e limites
        self.restricoes = {
            "peso_maximo": {
                "rodoviario": 30000,  # 30 toneladas
                "aereo": 500,         # 500 kg
                "fluvial": 50000      # 50 toneladas
            },
            "dimensoes_maximas": {
                "rodoviario": {"altura": 450, "largura": 245, "comprimento": 1800},  # cm
                "aereo": {"altura": 160, "largura": 150, "comprimento": 300},
                "fluvial": {"altura": 500, "largura": 300, "comprimento": 2000}
            }
        }

        # Adicionar custos de seguro por tipo de carga
        self.custos_seguro = {
            "comum": 0.001,      # 0.1% do valor declarado
            "frágil": 0.003,     # 0.3% do valor declarado
            "perecível": 0.005,  # 0.5% do valor declarado
            "perigosa": 0.01     # 1.0% do valor declarado
        }

        # Modalidades de transporte
        self.modalidades_transporte = {
            "rodoviario": {
                "custo_km": 1.0,
                "tempo_medio_km": 0.012,  # ~12h/1000km
                "peso_maximo": 30000,
                "disponivel_regioes": ["todas"]
            },
            "aereo": {
                "custo_km": 3.5,
                "tempo_medio_km": 0.001,  # ~1h/1000km
                "peso_maximo": 500,
                "disponivel_regioes": ["capitais"]
            },
            "fluvial": {
                "custo_km": 0.8,
                "tempo_medio_km": 0.024,  # ~24h/1000km
                "peso_maximo": 50000,
                "disponivel_regioes": ["Norte"]
            }
        }
        # Cache para resultados de cálculos frequentes
        self.cache_distancias: Dict[str, float] = {}
        self.cache_resultados: Dict[str, Dict[str, Any]] = {}
        
        # Tempo de expiração do cache (em segundos)
        self.cache_expiracao = 3600  # 1 hora
        self.cache_timestamp: Dict[str, float] = {}
    
    def calcular_distancia_real(self, origem: Tuple[float, float], destino: Tuple[float, float]) -> Optional[float]:
        """
        Calcula a distância real usando OSRM
        """
        try:
            # Inverte lat/long para formato do OSRM (long,lat)
            coords = f"{origem[1]},{origem[0]};{destino[1]},{destino[0]}"
            url = f"{self.osrm_url}/{coords}"
            
            response = requests.get(url, timeout=5)
            response.raise_for_status()
            
            data = response.json()
            if data["code"] != "Ok":
                self.logger.warning(f"Rota não encontrada para {origem} -> {destino}")
                return None
                
            distancia = data["routes"][0]["distance"] / 1000  # Converte para km
            return round(distancia, 2)
            
        except Exception as e:
            self.logger.warning(f"Erro ao calcular distância real: {str(e)}")
            return None

    def _gerar_chave_cache(self, 
                          origem: str, 
                          destino: str, 
                          peso: float, 
                          prazo: str, 
                          tipo_carga: str) -> str:
        """Gera uma chave única para o cache"""
        return f"{origem}|{destino}|{peso}|{prazo}|{tipo_carga}"

    def _cache_expirado(self, chave: str) -> bool:
        """Verifica se o cache expirou"""
        if chave not in self.cache_timestamp:
            return True
        tempo_atual = time.time()
        return (tempo_atual - self.cache_timestamp[chave]) > self.cache_expiracao

    def obter_distancia(self, origem: str, destino: str) -> Tuple[Optional[float], str]:
        """
        Obtém a distância entre duas cidades, retornando a distância e o tipo
        """
        self.logger.info(f"Calculando distância entre {origem} e {destino}")
        
        coord_origem = self.obter_coordenadas(origem)
        coord_destino = self.obter_coordenadas(destino)

        if not coord_origem or not coord_destino:
            self.logger.warning(f"Não foi possível obter coordenadas para {origem} ou {destino}")
            return (None, "erro")

        # Tenta primeiro obter distância real via OSRM
        distancia_real = self.calcular_distancia_real(coord_origem, coord_destino)
        
        if distancia_real is not None:
            self.logger.info(f"Distância real OSRM: {distancia_real:.2f} km")
            return (distancia_real, "real")
        
        # Se não conseguir, usa distância geodésica como fallback
        dist_geodesica = geodesic(coord_origem, coord_destino).kilometers
        self.logger.info(f"Usando distância geodésica: {dist_geodesica:.2f} km")
        return (round(dist_geodesica, 2), "geodésica")


   


    def validar_dimensoes(self, 
                         modalidade: str, 
                         altura: float, 
                         largura: float, 
                         comprimento: float) -> bool:
        """Valida se as dimensões são aceitas para a modalidade"""
        if not all([altura, largura, comprimento]):
            return True  # Se não informou dimensões, considera válido
            
        limites = self.restricoes["dimensoes_maximas"][modalidade]
        return (altura <= limites["altura"] and 
                largura <= limites["largura"] and 
                comprimento <= limites["comprimento"])

    def calcular_valor_seguro(self, 
                            tipo_carga: str, 
                            valor_mercadoria: float) -> float:
        """Calcula o valor do seguro baseado no tipo de carga"""
        return valor_mercadoria * self.custos_seguro.get(tipo_carga, 0.001)

    def calcular_prazo_entrega(self, 
                              distancia: float, 
                              modalidade: str, 
                              prazo: str) -> Dict[str, Any]:
        """Calcula o prazo estimado de entrega"""
        tempo_base = distancia * self.modalidades_transporte[modalidade]["tempo_medio_km"]
        
        # Ajusta baseado no tipo de prazo
        multiplicador = {
            "normal": 1.0,
            "expresso": 0.7,
            "urgente": 0.5
        }.get(prazo, 1.0)
        
        tempo_final = tempo_base * multiplicador
        
        # Converte para dias e horas
        dias = int(tempo_final // 24)
        horas = int((tempo_final % 24) * 60) // 60
        
        return {
            "dias": dias,
            "horas": horas,
            "prazo_tipo": prazo
        }

    def obter_coordenadas(self, cidade: str) -> Optional[Tuple[float, float]]:
        """
        Obtém as coordenadas geográficas de uma cidade com retry e timeout aumentado
        """
        if cidade in self.cache_coordenadas:
            return self.cache_coordenadas[cidade]
        
        try:
            # Aumenta timeout para 5 segundos e configura retries
            self.geocoder = Nominatim(
                user_agent="minha_calculadora_frete",
                timeout=5
            )
            
            # Adiciona retry com backoff exponencial
            for tentativa in range(3):  # 3 tentativas
                try:
                    location = self.geocoder.geocode(f"{cidade}, Brasil")
                    if location:
                        coordenadas = (location.latitude, location.longitude)
                        self.cache_coordenadas[cidade] = coordenadas
                        time.sleep(1)  # Respeita limites de API
                        return coordenadas
                    
                    time.sleep(2 ** tentativa)  # Backoff exponencial
                except Exception as e:
                    if tentativa == 2:  # Última tentativa
                        raise e
                    time.sleep(2 ** tentativa)
                    continue
                    
            return None
            
        except Exception as e:
            self.logger.error(f"Erro ao obter coordenadas para {cidade}: {str(e)}")
            return None

    def calcular_fator_peso(self, peso: float) -> float:
        """Calcula o fator multiplicador baseado no peso"""
        if peso <= 10:
            return 1.0
        elif peso <= 50:
            return 1.3
        elif peso <= 100:
            return 1.6
        else:
            return 2.0

    def calcular_fator_dimensao(self, volume: float) -> float:
        """Calcula o fator baseado no volume em m³"""
        if volume <= 0.125:
            return 1.0
        elif volume <= 0.5:
            return 1.3
        elif volume <= 2:
            return 1.6
        else:
            return 2.0

    def calcular_volume(self, altura: float, largura: float, comprimento: float) -> Optional[float]:
        """Calcula o volume em m³"""
        if all([altura, largura, comprimento]):
            return round((altura * largura * comprimento) / 1000000, 3)
        return None

    def obter_regiao(self, cidade: str) -> str:
        """Determina a região do Brasil baseada na cidade"""
        return self.mapa_regioes.get(cidade, "Sudeste")

    def calcular_peso_cubado(self, altura: float, largura: float, comprimento: float) -> float:
        """
        Calcula o peso cubado baseado nas dimensões (em cm)
        Peso cubado = (altura * largura * comprimento) / fator_cubagem
        """
        if all([altura, largura, comprimento]):
            return (altura * largura * comprimento) / self.fator_cubagem
        return 0.0

    def calcular_desconto_distancia(self, distancia: float) -> float:
        """
        Calcula o desconto baseado na distância usando faixas progressivas
        """
        # Garantir que as faixas sejam verificadas em ordem crescente
        for faixa in sorted(self.faixas_desconto_distancia.keys()):
            if distancia <= faixa:
                return self.faixas_desconto_distancia[faixa]
        return self.faixas_desconto_distancia[float('inf')]



    def calcular_custo_pedagio(self, distancia: float, regiao: str) -> float:
        """Calcula o custo estimado de pedágio para a rota"""
        return distancia * self.custos_pedagio.get(regiao, 0.10)

    def obter_fator_sazonal(self) -> float:
        """Retorna o fator sazonal baseado no mês atual"""
        mes_atual = datetime.now().month
        return self.fatores_sazonais.get(mes_atual, 1.0)

    def selecionar_modalidade_transporte(self, 
                                   distancia: float, 
                                   peso: float, 
                                   prazo: str,
                                   regiao_destino: str) -> Dict[str, Any]:
        """
        Seleciona sempre o modal rodoviário
        """
        return {
            "tipo": "rodoviario",
            "custo_km": self.modalidades_transporte["rodoviario"]["custo_km"]
        }

    def calcular_frete(self,
                      origem: str,
                      destino: str,
                      tipo_carga: str = "comum",
                      peso: float = 10.0,
                      prazo: str = "normal",
                      num_eixos: int = 3,
                      tipo_tabela: TipoTabela = TipoTabela.LOTACAO,
                      altura: float = 0,
                      largura: float = 0,
                      comprimento: float = 0,
                      valor_mercadoria: float = 0,
                      ignorar_cache: bool = False) -> Optional[Dict[str, Any]]:
        """Calcula o frete usando tabela ANTT como base"""
        try:
            # Gera chave de cache
            chave_cache = self._gerar_chave_cache(origem, destino, peso, prazo, tipo_carga)
            
            # Verifica cache
            if not ignorar_cache and chave_cache in self.cache_resultados:
                if not self._cache_expirado(chave_cache):
                    return self.cache_resultados[chave_cache]

            # Obtém distância
            distancia_info = self.obter_distancia(origem, destino)
            if distancia_info[0] is None:
                raise ValueError("Não foi possível calcular a distância")
            
            distancia, tipo_distancia = distancia_info

            # Obtém tipo de carga ANTT
            tipo_carga_antt = self.mapa_tipo_carga_antt.get(tipo_carga.lower())
            if not tipo_carga_antt:
                raise ValueError(f"Tipo de carga não mapeado: {tipo_carga}")

            # Calcula frete base ANTT
            frete_base = self.tabela_antt.calcular_frete_base(
                tipo_tabela,
                tipo_carga_antt,
                num_eixos,
                distancia
            )
            if not frete_base:
                raise ValueError("Não foi possível calcular frete base ANTT")

            # Aplica fatores adicionais
            peso_cubado = self.calcular_peso_cubado(altura, largura, comprimento)
            peso_considerado = max(peso, peso_cubado)

            # Calcula fatores
            fator_peso = self.calcular_fator_peso(peso_considerado)
            fator_prazo = self.fatores_prazo.get(prazo.lower(), 1.0)
            fator_tipo_carga = self.fatores_tipo_carga.get(tipo_carga.lower(), 1.0)
            
            # Regiões e fatores regionais
            regiao_origem = self.obter_regiao(origem)
            regiao_destino = self.obter_regiao(destino)
            fator_regiao = max(
                self.fatores_regiao[regiao_origem],
                self.fatores_regiao[regiao_destino]
            )

            # Custos adicionais
            custo_pedagio = self.calcular_custo_pedagio(distancia, regiao_destino)
            valor_seguro = self.calcular_valor_seguro(tipo_carga, valor_mercadoria)
            fator_sazonal = self.obter_fator_sazonal()
            
            # Calcula valor final
            valor_final = (frete_base * fator_peso * fator_prazo * 
                         fator_tipo_carga * fator_regiao * fator_sazonal +
                         custo_pedagio + valor_seguro)

            # Monta resultado
            resultado = {
                "origem": origem,
                "destino": destino,
                "distancia_km": round(distancia, 2),
                "tipo_distancia": tipo_distancia,
                "frete_base_antt": round(frete_base, 2),
                "peso_real": peso,
                "peso_cubado": round(peso_cubado, 2),
                "peso_considerado": round(peso_considerado, 2),
                "fatores_aplicados": {
                    "peso": fator_peso,
                    "prazo": fator_prazo,
                    "tipo_carga": fator_tipo_carga,
                    "regiao": fator_regiao,
                    "sazonal": fator_sazonal
                },
                "custos_adicionais": {
                    "pedagio": round(custo_pedagio, 2),
                    "seguro": round(valor_seguro, 2)
                },
                "valor_final": round(valor_final, 2)
            }

            # Atualiza cache
            self.cache_resultados[chave_cache] = resultado
            self.cache_timestamp[chave_cache] = time.time()

            return resultado

        except Exception as e:
            self.logger.error(f"Erro no cálculo do frete: {str(e)}")
            return None
        

    def processar_planilha(self, 
                        caminho_entrada: str, 
                        caminho_saida: str,
                        batch_size: int = 100) -> None:
        """
        Processa uma planilha Excel com processamento em lotes
        """
        try:
            if not os.path.exists(caminho_entrada):
                raise FileNotFoundError(f"Arquivo não encontrado: {caminho_entrada}")

            df = pd.read_excel(caminho_entrada)
            
            # Validação de colunas
            colunas_obrigatorias = ['origem', 'destino', 'tipo_tabela', 'num_eixos']
            if not all(col in df.columns for col in colunas_obrigatorias):
                raise ValueError(f"Colunas obrigatórias necessárias: {colunas_obrigatorias}")
            
            resultados = []
            total_rows = len(df)
            
            # Processar em lotes
            for i in range(0, total_rows, batch_size):
                batch = df.iloc[i:min(i+batch_size, total_rows)]
                batch_resultados = []
                
                for _, row in batch.iterrows():
                    # Converte tipo_tabela para enum
                    tipo_tabela = TipoTabela(row['tipo_tabela'])
                    
                    resultado = self.calcular_frete(
                        origem=row['origem'],
                        destino=row['destino'],
                        tipo_carga=str(row.get('tipo_carga', 'comum')),
                        peso=float(row.get('peso', 10.0)),
                        prazo=str(row.get('prazo', 'normal')),
                        num_eixos=int(row['num_eixos']),
                        tipo_tabela=tipo_tabela,
                        altura=float(row.get('altura', 0)),
                        largura=float(row.get('largura', 0)),
                        comprimento=float(row.get('comprimento', 0)),
                        valor_mercadoria=float(row.get('valor_mercadoria', 0))
                    )
                    
                    if resultado:
                        batch_resultados.append(resultado)
                    else:
                        batch_resultados.append({
                            'origem': row['origem'],
                            'destino': row['destino'],
                            'erro': 'Não foi possível calcular o frete'
                        })
                
                resultados.extend(batch_resultados)
                self.logger.info(f"Processado {min(i+batch_size, total_rows)} de {total_rows} registros")
            
            # Salvar resultados
            df_resultado = pd.DataFrame(resultados)
            df_resultado.to_excel(caminho_saida, index=False)
            self.logger.info(f"Resultados salvos em {caminho_saida}")
            
        except Exception as e:
            self.logger.error(f"Erro ao processar planilha: {str(e)}")
            raise

    def limpar_cache(self) -> None:
        """Limpa o cache de resultados e distâncias"""
        self.cache_distancias.clear()
        self.cache_resultados.clear()
        self.cache_timestamp.clear()






In [9]:
import pandas as pd
from typing import Dict, Any

def main() -> None:
    """
    Função principal para execução do programa
    """
    try:
        # Criar dados de teste
        dados_teste = {
            'origem': ['São Paulo', 'Curitiba', 'Porto Alegre', 'São Paulo'],
            'destino': ['Rio de Janeiro', 'Salvador', 'Manaus', 'Belém'],
            'peso': [10, 50, 100, 200],
            'prazo': ['normal', 'expresso', 'urgente', 'normal'],
            'tipo_carga': ['comum', 'frágil', 'perigosa', 'comum'],
            'altura': [20, 30, 40, 50],
            'largura': [30, 40, 50, 60],
            'comprimento': [40, 50, 60, 70],
            'valor_mercadoria': [1000, 5000, 10000, 20000],
            'tipo_tabela': ['A', 'A', 'A', 'A'],  # Tipo de tabela ANTT
            'num_eixos': [3, 4, 5, 6]  # Número de eixos
        }
        df = pd.DataFrame(dados_teste)
        df.to_excel('dados_entrada.xlsx', index=False)
        print("Arquivo de entrada criado com sucesso!")

        # Processar cálculos
        calculadora = CalculadoraFrete()
        calculadora.processar_planilha("dados_entrada.xlsx", "resultados.xlsx")
        
        print("Processamento concluído com sucesso!")
        
    except Exception as e:
        print(f"Erro durante a execução: {str(e)}")
    finally:
        print("\nPrograma finalizado!")

if __name__ == "__main__":
    main()

INFO:__main__:Calculando distância entre São Paulo e Rio de Janeiro


Arquivo de entrada criado com sucesso!


INFO:__main__:Distância real OSRM: 442.77 km
INFO:__main__:Calculando distância entre Curitiba e Salvador
INFO:__main__:Distância real OSRM: 2358.98 km
INFO:__main__:Calculando distância entre Porto Alegre e Manaus
INFO:__main__:Distância real OSRM: 4329.69 km
ERROR:__main__:Erro no cálculo do frete: Não foi possível calcular frete base ANTT
INFO:__main__:Calculando distância entre São Paulo e Belém
INFO:__main__:Distância real OSRM: 2866.21 km
INFO:__main__:Processado 4 de 4 registros
INFO:__main__:Resultados salvos em resultados.xlsx


Processamento concluído com sucesso!

Programa finalizado!


In [11]:
# import requests
# import pandas as pd
# from io import BytesIO
# import logging

# # Configuração do logger
# logging.basicConfig(level=logging.INFO)
# logger = logging.getLogger(__name__)

# def extrair_tabelas_resolucao() -> None:
#     """
#     Extrai tabelas da resolução ANTT e salva em arquivo Excel
#     """
#     try:
#         # URL da página com a resolução
#         url = "https://www.in.gov.br/web/dou/-/resolucao-n-6.046-de-11-de-julho-de-2024-571717582"
        
#         # Realiza o download do HTML da página
#         logger.info("Baixando conteúdo da página...")
#         response = requests.get(url, timeout=10)
#         response.raise_for_status()
        
#         # Extrai as tabelas
#         logger.info("Extraindo tabelas do HTML...")
#         tables = pd.read_html(
#             response.text,
#             decimal=",",
#             thousands=".",
#             encoding="utf-8"
#         )
        
#         if not tables:
#             raise ValueError("Nenhuma tabela encontrada na página")
            
#         # Salva as tabelas em Excel
#         logger.info(f"Salvando {len(tables)} tabelas encontradas...")
#         with pd.ExcelWriter("tabelas_resolucao_6046.xlsx", engine="openpyxl") as writer:
#             for i, df in enumerate(tables, start=1):
#                 # Limpa dados
#                 df.dropna(how="all", axis=1, inplace=True)
#                 df.dropna(how="all", axis=0, inplace=True)
                
#                 # Salva tabela
#                 sheet_name = f"Tabela_{i}"
#                 df.to_excel(writer, sheet_name=sheet_name, index=False)
#                 logger.info(f"Tabela {i} salva com sucesso")
        
#         logger.info("Processo concluído com sucesso!")
        
#     except requests.RequestException as e:
#         logger.error(f"Erro ao acessar a URL: {str(e)}")
#         raise
#     except ValueError as e:
#         logger.error(f"Erro ao processar dados: {str(e)}")
#         raise
#     except Exception as e:
#         logger.error(f"Erro inesperado: {str(e)}")
#         raise

# if __name__ == "__main__":
#     try:
#         extrair_tabelas_resolucao()
#     except Exception as e:
#         logger.error("Falha na execução do programa")
#         raise

In [28]:
def processar_tabela_antt(df: pd.DataFrame) -> Dict[int, Dict[int, CoeficientesANTT]]:
    """
    Processa o DataFrame da tabela ANTT e retorna um dicionário formatado
    """
    resultado = {}
    
    # Preenche valores NaN no tipo de carga com o último valor válido
    df['Tipo de carga'] = df['Tipo de carga'].ffill()
    
    # Mapeia os tipos de carga para os valores do enum
    mapa_tipos = {
        "Granel sólido": TipoCargaANTT.GRANEL_SOLIDO.value,
        "Granel líquido": TipoCargaANTT.GRANEL_LIQUIDO.value,
        "Frigorificada ou Aquecida": TipoCargaANTT.FRIGORIFICADA.value,
        "Conteinerizada": TipoCargaANTT.CONTEINERIZADA.value,
        "Carga Geral": TipoCargaANTT.CARGA_GERAL.value,
        "Neogranel": TipoCargaANTT.NEOGRANEL.value,
        "Perigosa (granel sólido)": TipoCargaANTT.PERIGOSA_GRANEL_SOLIDO.value,
        "Perigosa (granel líquido)": TipoCargaANTT.PERIGOSA_GRANEL_LIQUIDO.value,
        "Perigosa (frigorificada ou aquecida)": TipoCargaANTT.PERIGOSA_FRIGORIFICADA.value,
        "Perigosa (conteinerizada)": TipoCargaANTT.PERIGOSA_CONTEINERIZADA.value,
        "Perigosa (carga geral)": TipoCargaANTT.PERIGOSA_CARGA_GERAL.value,
        "Carga Granel Pressurizada": TipoCargaANTT.GRANEL_PRESSURIZADA.value
    }
    
    # Obtém os números de eixos da primeira linha
    colunas_eixos = [col for col in df.columns if "Número de eixos carregados" in col]
    num_eixos = []
    
    if len(colunas_eixos) > 0:
        primeira_linha = df.iloc[0]
        for col in colunas_eixos:
            if pd.notna(primeira_linha[col]):
                num_eixos.append((col, int(primeira_linha[col])))
    
    print(f"Números de eixos encontrados: {[x[1] for x in num_eixos]}")
    
    # Processa cada tipo de carga
    tipo_carga_atual = None
    coeficientes = {"deslocamento": None, "carga": None}
    
    for idx, row in df.iterrows():
        if pd.notna(row['Coeficiente de custo']):
            coef_custo = str(row['Coeficiente de custo']).lower()
            
            if "deslocamento" in coef_custo:
                coeficientes["deslocamento"] = row
                print(f"Encontrado deslocamento para: {row['Tipo de carga']}")
                
            elif "carga" in coef_custo:
                coeficientes["carga"] = row
                print(f"Encontrado carga para: {row['Tipo de carga']}")
                
                # Processa os coeficientes quando tiver tanto deslocamento quanto carga
                if coeficientes["deslocamento"] is not None:
                    tipo_carga = mapa_tipos.get(row['Tipo de carga'])
                    if tipo_carga is not None:
                        resultado[tipo_carga] = {}
                        
                        for col, num_eixo in num_eixos:
                            desl = coeficientes["deslocamento"][col]
                            carga = coeficientes["carga"][col]
                            
                            if pd.notna(desl) and pd.notna(carga):
                                resultado[tipo_carga][num_eixo] = CoeficientesANTT(
                                    deslocamento=float(desl),
                                    carga_descarga=float(carga)
                                )
                                print(f"Processado {row['Tipo de carga']} - {num_eixo} eixos")
                    
                    # Reseta os coeficientes para o próximo tipo de carga
                    coeficientes = {"deslocamento": None, "carga": None}
    
    # Validação final
    if not resultado:
        raise ValueError("Nenhum dado foi processado com sucesso")
    
    print(f"\nTipos de carga processados: {len(resultado)}")
    for tipo_carga, dados in resultado.items():
        print(f"Tipo {tipo_carga}: {len(dados)} configurações de eixos")
    
    return resultado

def validar_dados_processados(resultado: Dict[int, Dict[int, CoeficientesANTT]]) -> None:
    """
    Valida os dados processados e gera relatório detalhado
    """
    print("\nRelatório de Validação:")
    
    # Verifica valores negativos ou zero
    for tipo_carga, configs in resultado.items():
        for num_eixos, coef in configs.items():
            if coef.deslocamento <= 0 or coef.carga_descarga <= 0:
                print(f"ALERTA: Valores inválidos para tipo {tipo_carga}, {num_eixos} eixos")
                print(f"  Deslocamento: {coef.deslocamento}")
                print(f"  Carga/Descarga: {coef.carga_descarga}")
    
    # Verifica consistência dos valores
    for tipo_carga, configs in resultado.items():
        eixos = sorted(configs.keys())
        for i in range(len(eixos)-1):
            atual = configs[eixos[i]]
            proximo = configs[eixos[i+1]]
            if atual.deslocamento >= proximo.deslocamento:
                print(f"ALERTA: Inconsistência nos valores de deslocamento para tipo {tipo_carga}")
                print(f"  {eixos[i]} eixos: {atual.deslocamento}")
                print(f"  {eixos[i+1]} eixos: {proximo.deslocamento}")

    print("\nValidação concluída!")

def atualizar_tabela_antt(caminho_arquivo: str) -> TabelaANTT:
    """
    Atualiza a tabela ANTT com dados do arquivo Excel
    """
    try:
        # Lê o arquivo Excel
        df = pd.read_excel(caminho_arquivo)
        print("\nPrimeiras linhas da tabela:")
        print(df.head())
        
        # Processa a tabela
        dados_formatados = processar_tabela_antt(df)
        
        # Atualiza a classe TabelaANTT
        tabela = TabelaANTT()
        tabela.tabelas[TipoTabela.LOTACAO] = dados_formatados
        
        return tabela
        
    except Exception as e:
        logging.error(f"Erro ao processar tabela ANTT: {str(e)}")
        raise

In [29]:
# Exemplo de uso
tabela_atualizada = atualizar_tabela_antt("/home/romulobrito/projetos/fretes/tabelas_resolucao_6046.xlsx")

# Usar a tabela atualizada na calculadora
calculadora = CalculadoraFrete()
calculadora.tabela_antt = tabela_atualizada


Primeiras linhas da tabela:
    Tipo de carga   Coeficiente de custo unidade  \
0             NaN                    NaN     NaN   
1   Granel sólido     Deslocamento (CCD)   R$/km   
2             NaN  Carga e descarga (CC)      R$   
3  Granel líquido     Deslocamento (CCD)   R$/km   
4             NaN  Carga e descarga (CC)      R$   

   Número de eixos carregados do veículo combinado  \
0                                           2.0000   
1                                           3.4880   
2                                         408.5700   
3                                           3.5634   
4                                         423.6500   

   Número de eixos carregados do veículo combinado.1  \
0                                             3.0000   
1                                             4.4505   
2                                           495.6000   
3                                             4.5518   
4                                           517.8100 

In [36]:
from dataclasses import dataclass
from enum import Enum
from typing import Dict, Optional
import pandas as pd
import logging
from geopy.distance import geodesic
from geopy.geocoders import Nominatim
import requests
import time

@dataclass
class CoeficienteANTT:
    """Coeficientes de custo conforme tabela ANTT"""
    deslocamento: float  # CCD - R$/km
    carga_descarga: float  # CC - R$


def criar_tabela_antt():
    """
    Cria uma tabela ANTT estruturada para fácil leitura
    """
    # Estrutura da tabela
    dados = {
        'tipo_carga': [
            'Granel sólido', 'Granel sólido', 'Granel sólido', 'Granel sólido', 'Granel sólido', 'Granel sólido', 'Granel sólido',
            'Granel líquido', 'Granel líquido', 'Granel líquido', 'Granel líquido', 'Granel líquido', 'Granel líquido', 'Granel líquido',
            'Frigorificada', 'Frigorificada', 'Frigorificada', 'Frigorificada', 'Frigorificada', 'Frigorificada', 'Frigorificada',
            'Conteinerizada', 'Conteinerizada', 'Conteinerizada', 'Conteinerizada', 'Conteinerizada', 'Conteinerizada',
            'Carga Geral', 'Carga Geral', 'Carga Geral', 'Carga Geral', 'Carga Geral', 'Carga Geral', 'Carga Geral'
        ],
        'num_eixos': [
            2, 3, 4, 5, 6, 7, 9,
            2, 3, 4, 5, 6, 7, 9,
            2, 3, 4, 5, 6, 7, 9,
            3, 4, 5, 6, 7, 9,
            2, 3, 4, 5, 6, 7, 9
        ],
        'coef_deslocamento': [  # CCD - R$/km
            3.4880, 4.4505, 5.1405, 5.5749, 6.2063, 7.0506, 7.9437,
            3.5634, 4.5518, 5.1213, 5.7255, 6.4428, 7.1940, 8.2354,
            4.1977, 5.3292, 6.1395, 6.7886, 7.4860, 8.9162, 9.8118,
            4.4235, 5.0137, 5.5159, 6.1318, 7.0371, 7.8991,
            3.4645, 4.4360, 5.0845, 5.5455, 6.1249, 7.0577, 7.9940
        ],
        'coef_carga_descarga': [  # CC - R$
            408.57, 495.60, 536.30, 520.16, 546.66, 710.36, 755.95,
            423.65, 517.81, 520.16, 550.70, 600.82, 738.92, 825.27,
            490.43, 589.50, 640.20, 658.43, 675.05, 999.88, 1011.88,
            488.19, 501.44, 503.95, 526.18, 706.64, 743.67,
            402.10, 491.61, 520.92, 512.07, 524.27, 712.31, 769.78
        ]
    }
    
    # Cria o DataFrame
    df = pd.DataFrame(dados)
    
    # Salva em Excel
    df.to_excel('tabela_antt_estruturada.xlsx', index=False)
    print("Tabela ANTT estruturada criada com sucesso!")
    
    return df

def processar_tabela_antt(df: pd.DataFrame) -> Dict[str, Dict[int, CoeficienteANTT]]:
    """
    Processa a tabela ANTT estruturada e retorna um dicionário com os coeficientes
    """
    coeficientes = {}
    
    # Processa cada linha da tabela
    for _, row in df.iterrows():
        tipo_carga = row['tipo_carga']
        num_eixos = int(row['num_eixos'])
        
        if tipo_carga not in coeficientes:
            coeficientes[tipo_carga] = {}
            
        coeficientes[tipo_carga][num_eixos] = CoeficienteANTT(
            deslocamento=float(row['coef_deslocamento']),
            carga_descarga=float(row['coef_carga_descarga'])
        )
    
    # Validação e log
    print("\nTipos de carga processados:")
    for tipo_carga, configs in coeficientes.items():
        print(f"{tipo_carga}: {len(configs)} configurações de eixos")
        
    return coeficientes


class CalculadoraFreteANTT:
    def __init__(self, tabela_antt: pd.DataFrame) -> None:
        """Inicializa a calculadora com a tabela ANTT"""
        self.logger = logging.getLogger(__name__)
        self.geocoder = Nominatim(user_agent="calculadora_frete")
        self.coeficientes = processar_tabela_antt(tabela_antt)
    
    def calcular_distancia(self, origem: str, destino: str) -> Optional[float]:
        """Calcula a distância entre origem e destino"""
        try:
            loc_origem = self.geocoder.geocode(f"{origem}, Brasil")
            time.sleep(1)  # Respeita limites da API
            loc_destino = self.geocoder.geocode(f"{destino}, Brasil")
            
            if loc_origem and loc_destino:
                distancia = geodesic(
                    (loc_origem.latitude, loc_origem.longitude),
                    (loc_destino.latitude, loc_destino.longitude)
                ).kilometers
                return round(distancia, 2)
            return None
        except Exception as e:
            self.logger.error(f"Erro ao calcular distância: {str(e)}")
            return None
    
    def calcular_frete(self, 
                      origem: str, 
                      destino: str, 
                      tipo_carga: str,
                      num_eixos: int) -> Optional[Dict]:
        """
        Calcula o frete usando os coeficientes ANTT
        Frete = CC + (CCD * distância)
        """
        try:
            # Verifica se existe coeficiente para o tipo de carga e número de eixos
            if tipo_carga not in self.coeficientes or num_eixos not in self.coeficientes[tipo_carga]:
                raise ValueError(f"Combinação inválida: {tipo_carga} com {num_eixos} eixos")

            # Calcula distância
            distancia = self.calcular_distancia(origem, destino)
            if not distancia:
                raise ValueError("Não foi possível calcular a distância")

            # Obtém coeficientes
            coef = self.coeficientes[tipo_carga][num_eixos]

            # Calcula frete
            valor_frete = coef.carga_descarga + (coef.deslocamento * distancia)

            return {
                "origem": origem,
                "destino": destino,
                "tipo_carga": tipo_carga,
                "distancia_km": distancia,
                "num_eixos": num_eixos,
                "coeficientes": {
                    "deslocamento": coef.deslocamento,
                    "carga_descarga": coef.carga_descarga
                },
                "valor_frete": round(valor_frete, 2)
            }

        except Exception as e:
            self.logger.error(f"Erro no cálculo: {str(e)}")
            return None


def main():
    """Função principal"""
    try:
        # Cria e lê a tabela ANTT estruturada
        tabela_antt = criar_tabela_antt()
        
        # Cria dados de teste
        dados_teste = {
            'origem': ['São Paulo', 'Curitiba', 'Porto Alegre'],
            'destino': ['Rio de Janeiro', 'Salvador', 'Manaus'],
            'tipo_carga': ['Granel sólido', 'Granel líquido', 'Carga Geral'],
            'num_eixos': [3, 4, 5]
        }
        
        # Cria arquivo de entrada
        df_entrada = pd.DataFrame(dados_teste)
        df_entrada.to_excel('dados_entrada.xlsx', index=False)
        print("\nArquivo de entrada criado com sucesso!")
        
        # Processa cálculos
        calculadora = CalculadoraFreteANTT(tabela_antt)
        resultados = []
        
        for _, row in df_entrada.iterrows():
            resultado = calculadora.calcular_frete(
                origem=row['origem'],
                destino=row['destino'],
                tipo_carga=row['tipo_carga'],
                num_eixos=row['num_eixos']
            )
            resultados.append(resultado or {
                'erro': 'Falha no cálculo',
                **row.to_dict()
            })
        
        # Salva resultados
        df_resultados = pd.DataFrame(resultados)
        df_resultados.to_excel('resultados.xlsx', index=False)
        print("Processamento concluído com sucesso!")
        
    except Exception as e:
        print(f"Erro durante a execução: {str(e)}")

if __name__ == "__main__":
    main()

Tabela ANTT estruturada criada com sucesso!

Arquivo de entrada criado com sucesso!

Tipos de carga processados:
Granel sólido: 7 configurações de eixos
Granel líquido: 7 configurações de eixos
Frigorificada: 7 configurações de eixos
Conteinerizada: 6 configurações de eixos
Carga Geral: 7 configurações de eixos


Processamento concluído com sucesso!
